In [23]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor

# train data file load
# delete 'Index' because it is provided when converted to a data frame, and delete 'Standard_Weight' because it is determined by the hegith
file_df = pd.read_csv('./train_data.csv')
target_name = 'Body_Fat_Rate'
no_need_features = ['Index', 'Standard_Weight']
category_features = ['Sex']

# arrange X and y
file_df.drop(no_need_features, axis=1, inplace=True)
y_target = file_df[target_name]
X_features = file_df.drop([target_name],axis=1,inplace=False)

# visualize data to find outliers
outlier_name = 'Height'
cond1 = file_df[outlier_name] < 60
cond2 = file_df[target_name] < 30
outlier_index = X_features[cond1 & cond2].index
X_features.drop(outlier_index , axis=0, inplace=True)
y_target.drop(outlier_index, axis=0, inplace=True)

# change the category feature to One-Hot Encoding --> 'Sex'
X_features_ohe = pd.get_dummies(X_features, columns=category_features)

# single model
model = RandomForestRegressor(max_depth=14, min_samples_leaf=2, min_samples_split=2, n_estimators=700, n_jobs=-1)
model.fit(X_features_ohe, y_target)

# mixed model
# model1 = RandomForestRegressor(max_depth=14, min_samples_leaf=2, min_samples_split=2, n_estimators=700, n_jobs=-1)
# model2 = XGBRegressor(eta=0.1, min_child_weight=3, max_depth=3, n_estimators=120)
# model1.fit(X_features_ohe, y_target)
# model2.fit(X_features_ohe, y_target)

# test data file load
# delete 'Index' because it is provided when converted to a data frame, and delete 'Standard_Weight' because it is determined by the hegith
test_df = pd.read_csv('./test_data.csv')
# print(test_df)

# arrange X and y
test_df.drop(no_need_features, axis=1, inplace=True)
y_test = test_df[target_name]
X_test = test_df.drop([target_name],axis=1,inplace=False)

# change the category feature to One-Hot Encoding --> 'Sex'
X_test_ohe = pd.get_dummies(X_test, columns=category_features)

# single model
predict_value = model.predict(X_test_ohe)
print("**Single Model**")
for x in predict_value:
    print(round(x,1))

# mixed model
# pred1 = model1.predict(X_test_ohe)
# pred2 = model2.predict(X_test_ohe)
# pred = 0.8 * pred1 + 0.2 * pred2
# print("**Mixed Model**")
# for x in pred:
#     print(round(x,1))

**Single Model**
25.0
24.0
13.5
20.4
35.1
18.1
23.3
12.0
24.9
13.2
31.4
16.3
13.5
19.1
11.5
33.4
18.9
17.2
12.6
10.3
30.2
29.2
30.8
19.3
25.5
13.3
19.0
26.0
26.4
19.1
24.5
17.1
14.9
26.1
15.7
32.7
25.7
18.8
20.4
25.1
17.1
29.0
19.4
33.0
26.9
21.7
14.4
18.2
28.1
31.1
23.1
25.5
14.7
27.5
25.5
15.4
27.7
23.7
25.1
33.1
26.4
25.6
31.8
23.2
24.5
26.6
28.8
17.8
27.1
25.0
36.7
31.6
22.9
27.6
30.5
25.0
18.6
28.3
27.2
29.9
29.6
26.9
26.3
23.8
28.1
25.9
26.3
21.0
28.2
35.5
24.3
29.1
26.0
30.2
27.7
31.4
27.7
23.2
30.3
32.0
26.7
23.2
14.6
29.2
36.5
26.9
24.9
25.6
23.3
21.8
31.0
22.4
21.5
28.4
33.6
20.9
23.7
29.0
23.5
21.5
29.4
43.9
22.1
35.5
21.7
35.5
22.3
28.6
29.8
20.1
36.8
35.6
23.4
38.9
41.0
26.1
21.5
39.0
26.3
26.5
36.3
25.5
34.1
35.5
32.9
21.1
36.1
26.9
31.1
25.4
32.4
40.9
24.9
37.4
40.7
24.7
14.5
17.2
17.5
29.8
16.6
16.3
25.1
25.5
17.5
35.4
8.9
26.9
23.5
